# Construcción de Operaciones

Para ejecutar todos los ejemplos se debe importar la librería. Se sugiere utilizar siempre el alias `qcf`. 

In [1]:
import qcfinancial as qcf

# Para el despliegue de tablas de desarrollo, se importa pandas de la forma usual.
import pandas as pd

In [2]:
qcf.id()

'3'

El siguiente diccionario se utiliza para dar formato a las columnas de los `pandas.DataFrames`.

In [3]:
format_dict = {
    'nominal': '{0:,.2f}', 
    'nocional': '{0:,.2f}', 
    'amort': '{0:,.2f}', 
    'amortizacion': '{0:,.2f}', 
    'interes': '{0:,.2f}', 
    'flujo': '{0:,.2f}',
    'icp_inicial': '{0:,.2f}', 
    'icp_final': '{0:,.2f}',
    'valor_tasa': '{0:,.4%}', 
    'valor_tasa_equivalente': '{0:,.6%}', 
    'spread': '{0:,.4%}', 
    'gearing': '{0:,.2f}',
    'amort_moneda_pago': '{0:,.2f}', 
    'interes_moneda_pago': '{0:,.2f}', 
    'valor_indice_fx': '{0:,.2f}'
}

## Legs
Los objetos de tipo `Leg` son una lista (o vector) de objetos `Cashflow` y representan una pata de un instrumento financiero. un objeto de tipo `Leg` puede construirse *a mano* es decir, dando de alta cashflows y agregándolos uno a uno o con algunos métodos de conveniencia cuyo funcionamiento se mostrará.
### Construcción Manual
Se verá como construir un `Leg` con 2 `SimpleCashflow` de forma *manual*. En particular, este objeto `Leg` podría representar una operación FX por entrega física.

In [4]:
fecha_vcto = qcf.QCDate(20, 9, 2018)

simple_cashflow_1 = qcf.SimpleCashflow(
    fecha_vcto,  # fecha del flujo
    100,         # monto
    qcf.QCCLP()  # moneda
) 

simple_cashflow_2 = qcf.SimpleCashflow(
    fecha_vcto,  # fecha del flujo
    -70000,      # monto
    qcf.QCUSD()  # moneda
)

leg = qcf.Leg()
leg.append_cashflow(simple_cashflow_1)
leg.append_cashflow(simple_cashflow_2)

In [5]:
# Se observa el resultado
num = leg.size()
for i in range(0, num):
    print(qcf.show(leg.get_cashflow_at(i)))

('2018-09-20', 100.0, 'CLP')
('2018-09-20', -70000.0, 'USD')


### Construcción Asistida de un `FixedRateLeg`
Se verá como construir objetos `Leg` donde cada `Cashflow` es un objeto de tipo `FixedRateCashflow`, todos con la misma tasa fija. En el primer ejemplo se construye un `Leg` de tipo *bullet*: una única amortización igual al capital vigente de todos los `FixedRateCasflow` en el último flujo.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `float`: nominal inicial
- `bool`: si es `True` significa que la amortización es un flujo de caja efectivo
- `QCInterestRate`: la tasa a aplicar en cada flujo
- `QCCurrency`: moneda del nominal y de los flujos
- `bool`: si es `True` fuerza a que las fechas de pago coincidan con las fechas finales. Esto para lograr una valorización acorde a las convenciones de los mercados de renta fija, en caso que la `Leg` represente un bono a tasa fija.

Vamos a un ejemplo. Cambiando los parámetros siguientes se puede visualizar el efecto de ellos en la construcción.

In [6]:
# Se da de alta los parámetros requeridos
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(5, 11, 2019)
fecha_final = qcf.QCDate(31, 5, 2023)
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad = qcf.Tenor('6M')
periodo_irregular = qcf.StubPeriod.LONGFRONT
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
calendario.add_holiday(qcf.QCDate(31, 12, 2019))
lag_pago = 0
nominal = 100000.0
amort_es_flujo = False
tasa_cupon = qcf.QCInterestRate(.03, qcf.QCAct360(), qcf.QCLinearWf())
moneda = qcf.QCCLF()
es_bono = False

fixed_rate_leg = qcf.LegFactory.build_bullet_fixed_rate_leg(
    rec_pay=rp,
    start_date=fecha_inicio,
    end_date=fecha_final,
    bus_adj_rule=bus_adj_rule,
    settlement_periodicity=periodicidad,
    stub_period=periodo_irregular,
    settlement_calendar=calendario,
    settlement_lag=lag_pago,
    initial_notional=nominal,
    amort_is_cashflow=amort_es_flujo,
    interest_rate=tasa_cupon,
    notional_currency=moneda,
    is_bond=es_bono
)

Se puede lograr una visualización del resultado utilizando un Dataframe de pandas y el método `show`.

In [7]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, fixed_rate_leg.size()):
    tabla.append(qcf.show(fixed_rate_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_ini', 'fecha_fin', 'fecha_pago', 'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo', 'moneda',
            'valor_tasa', 'tipo_tasa']
df = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
# df.style.format(format_dict))
df

,fecha_ini,fecha_fin,fecha_pago,nominal,amort,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2019-11-05,2020-05-29,2020-05-29,100000.0,0.0,1716.666667,False,1716.666667,CLF,0.03,LinAct360
1,2020-05-29,2020-11-30,2020-11-30,100000.0,0.0,1541.666667,False,1541.666667,CLF,0.03,LinAct360
2,2020-11-30,2021-05-31,2021-05-31,100000.0,0.0,1516.666667,False,1516.666667,CLF,0.03,LinAct360
3,2021-05-31,2021-11-30,2021-11-30,100000.0,0.0,1525.000000,False,1525.000000,CLF,0.03,LinAct360
4,2021-11-30,2022-05-31,2022-05-31,100000.0,0.0,1516.666667,False,1516.666667,CLF,0.03,LinAct360
5,2022-05-31,2022-11-30,2022-11-30,100000.0,0.0,1525.000000,False,1525.000000,CLF,0.03,LinAct360
6,2022-11-30,2023-05-31,2023-05-31,100000.0,100000.0,1516.666667,False,1516.666667,CLF,0.03,LinAct360


### Construcción Asistida de un `FixedRateLeg2`
Se verá como construir objetos `Leg` donde cada `Cashflow` es un objeto de tipo `FixedRateCashflow2`, todos con la misma tasa fija. En el primer ejemplo se construye un `Leg` de tipo *bullet*: una única amortización igual al capital vigente de todos los `FixedRateCasflow2` en el último flujo.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `float`: nominal inicial
- `bool`: si es `True` significa que la amortización es un flujo de caja efectivo
- `QCInterestRate`: la tasa a aplicar en cada flujo
- `QCCurrency`: moneda del nominal y de los flujos
- `bool`: si es `True` fuerza a que las fechas de pago coincidan con las fechas finales. Esto para lograr una valorización acorde a las convenciones de los mercados de renta fija, en caso que la `Leg` represente un bono a tasa fija.

Vamos a un ejemplo. Cambiando los parámetros siguientes se puede visualizar el efecto de ellos en la construcción.

In [8]:
# Se da de alta los parámetros requeridos
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(4, 4, 2018)
fecha_final = qcf.QCDate(1, 3, 2021) 
bus_adj_rule = qcf.BusyAdjRules.NO
periodicidad = qcf.Tenor('6M')
periodo_irregular = qcf.StubPeriod.SHORTFRONT
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
calendario.add_holiday(qcf.QCDate(31, 12, 2018))
lag_pago = 0
nominal = 100000.0
amort_es_flujo = False
tasa_cupon = qcf.QCInterestRate(.03, qcf.QCAct360(), qcf.QCLinearWf())
moneda = qcf.QCCLF()
es_bono = False

# Se da de alta el objeto
fixed_rate_leg_2 = qcf.LegFactory.build_bullet_fixed_rate_leg_2(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad,
    periodo_irregular,
    calendario,
    lag_pago,
    nominal,
    amort_es_flujo,
    tasa_cupon,
    moneda,
    es_bono
)

In [9]:
tabla = []
for i in range(fixed_rate_leg_2.size()):
    tabla.append(qcf.show(fixed_rate_leg_2.get_cashflow_at(i)))
    
df2 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df2

,fecha_ini,fecha_fin,fecha_pago,nominal,amort,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2018-04-04,2018-09-01,2018-09-03,100000.0,0.0,1250.000000,False,1250.000000,CLF,0.03,LinAct360
1,2018-09-01,2019-03-01,2019-03-01,100000.0,0.0,1508.333333,False,1508.333333,CLF,0.03,LinAct360
2,2019-03-01,2019-09-01,2019-09-02,100000.0,0.0,1533.333333,False,1533.333333,CLF,0.03,LinAct360
3,2019-09-01,2020-03-01,2020-03-02,100000.0,0.0,1516.666667,False,1516.666667,CLF,0.03,LinAct360
4,2020-03-01,2020-09-01,2020-09-01,100000.0,0.0,1533.333333,False,1533.333333,CLF,0.03,LinAct360
5,2020-09-01,2021-03-01,2021-03-01,100000.0,100000.0,1508.333333,False,1508.333333,CLF,0.03,LinAct360


### Construcción Asistida de un `CustomAmortFixedRateLeg`
En este ejemplo se construye un `Leg` donde la estructura de amortizaciones es customizada.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `CustomNotionalAndAmort`: vector de capital vigente y amortizaciones customizado
- `bool`: si es `True` significa que la amortización es un flujo de caja efectivo
- `QCInterestRate`: la tasa a aplicar en cada flujo
- `QCCurrency`: moneda del nominal y de los flujos
- `bool`: si es `True` fuerza a que las fechas de pago coincidan con las fechas finales. Esto para lograr una valorización acorde a las convenciones de los mercados de renta fija, en caso que la `Leg` represente un bono a tasa fija.

Vamos a un ejemplo. Cambiando los parámetros siguientes se puede visualizar el efecto de ellos en la construcción.

In [10]:
# Se da de alta los parámetros requeridos
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(31, 1, 1969)
fecha_final = qcf.QCDate(31, 1, 1974) 
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad = qcf.Tenor('6M')
periodo_irregular = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
custom_notional_amort = qcf.CustomNotionalAmort()
custom_notional_amort.set_size(10)
for i in range(0, 10):
    custom_notional_amort.set_notional_amort_at(i, 1000.0 - i * 100.0, 100.0)
amort_es_flujo = False
tasa_cupon = qcf.QCInterestRate(.03, qcf.QC30360(), qcf.QCLinearWf())
moneda = qcf.QCCLF()
es_bono = False

In [11]:
# Se da de alta el objeto
fixed_rate_custom_leg = qcf.LegFactory.build_custom_amort_fixed_rate_leg(
    rec_pay=rp,
    start_date=fecha_inicio,
    end_date=fecha_final,
    bus_adj_rule=bus_adj_rule,
    settlement_periodicity=periodicidad,
    stub_period=periodo_irregular,
    settlement_calendar=calendario,
    settlement_lag=lag_pago,
    notional_and_amort=custom_notional_amort, # custom_notional_amort,
    amort_is_cashflow=amort_es_flujo,
    interest_rate=tasa_cupon,
    notional_currency=moneda
)

In [12]:
params = dict(
    rec_pay=rp,
    start_date=fecha_inicio,
    end_date=fecha_final,
    bus_adj_rule=bus_adj_rule,
    settlement_periodicity=periodicidad,
    stub_period=periodo_irregular,
    settlement_calendar=calendario,
    settlement_lag=lag_pago,
    notional_and_amort=custom_notional_amort, # custom_notional_amort,
    amort_is_cashflow=amort_es_flujo,
    interest_rate=tasa_cupon,
    notional_currency=moneda
)

In [13]:
fixed_rate_custom_leg = qcf.LegFactory.build_custom_amort_fixed_rate_leg(**params)

In [14]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, fixed_rate_custom_leg.size()):
    tabla.append(qcf.show(fixed_rate_custom_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_ini', 'fecha_fin', 'fecha_pago', 'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo', 'moneda',
            'valor_tasa', 'tipo_tasa']
df3 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df3 # .style.format(format_dict)

,fecha_ini,fecha_fin,fecha_pago,nominal,amort,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,1969-01-31,1969-07-31,1969-07-31,1000.0,100.0,15.000000,False,15.000000,CLF,0.03,Lin30360
1,1969-07-31,1970-01-30,1970-01-30,900.0,100.0,13.500000,False,13.500000,CLF,0.03,Lin30360
2,1970-01-30,1970-07-31,1970-07-31,800.0,100.0,12.000000,False,12.000000,CLF,0.03,Lin30360
3,1970-07-31,1971-01-29,1971-01-29,700.0,100.0,10.441667,False,10.441667,CLF,0.03,Lin30360
4,1971-01-29,1971-07-30,1971-07-30,600.0,100.0,9.050000,False,9.050000,CLF,0.03,Lin30360
5,1971-07-30,1972-01-31,1972-01-31,500.0,100.0,7.500000,False,7.500000,CLF,0.03,Lin30360
6,1972-01-31,1972-07-31,1972-07-31,400.0,100.0,6.000000,False,6.000000,CLF,0.03,Lin30360
7,1972-07-31,1973-01-31,1973-01-31,300.0,100.0,4.500000,False,4.500000,CLF,0.03,Lin30360
8,1973-01-31,1973-07-31,1973-07-31,200.0,100.0,3.000000,False,3.000000,CLF,0.03,Lin30360
9,1973-07-31,1974-01-31,1974-01-31,100.0,100.0,1.500000,False,1.500000,CLF,0.03,Lin30360


### Construcción Asistida de un `FixedRateMultiCurrencyLeg`
Se verá como construir objetos `Leg` donde cada `Cashflow` es un objeto de tipo `FixedRateMultiCurrencyCashflow`, todos con la misma tasa fija. En el primer ejemplo se construye un `Leg` de tipo *bullet*: una única amortización igual al capital vigente de todos los `FixedRateMultiCurrencyCasflow` en el último flujo.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `float`: nominal inicial
- `bool`: si es `True` significa que la amortización es un flujo de caja efectivo
- `QCInterestRate`: la tasa a aplicar en cada flujo
- `QCCurrency`: moneda del nominal
- `QCCurrency`: moneda de los flujos
- `FXRateIndex`: índice con el cual se transforma cada flujo a la moneda de pago.
- `bool`: si es `True` fuerza a que las fechas de pago coincidan con las fechas finales. Esto para lograr una valorización acorde a las convenciones de los mercados de renta fija, en caso que la `Leg` represente un bono a tasa fija.

Vamos a un ejemplo. Cambiando los parámetros siguientes se puede visualizar el efecto de ellos en la construcción.

In [15]:
# Primero se debe dar de alta un FXRateIndex
usd = qcf.QCUSD()
clp = qcf.QCCLP()
usdclp = qcf.FXRate(usd, clp)
one_d = qcf.Tenor('1D')
usdclp_obs = qcf.FXRateIndex(usdclp, 'USDOBS', one_d, one_d, calendario)

# Luego se dan de alta los otros parámetros requeridos para la construcción
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(31, 1, 1969)
fecha_final = qcf.QCDate(31, 1, 1974) 
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad = qcf.Tenor('6M')
periodo_irregular = qcf.StubPeriod.NO
lag_pago = 1
es_bono = False

In [16]:
# Se da de alta el objeto
fixed_rate_mccy_leg = qcf.LegFactory.build_bullet_fixed_rate_mccy_leg(
    rec_pay=rp,
    start_date=fecha_inicio,
    end_date=fecha_final,
    bus_adj_rule=bus_adj_rule,
    settlement_periodicity=periodicidad,
    stub_period=periodo_irregular,
    settlement_calendar=calendario,
    settlement_lag=lag_pago,
    initial_notional=nominal,
    amort_is_cashflow=amort_es_flujo,
    interest_rate=tasa_cupon,
    notional_currency=usd,
    settlement_currency=clp,
    fx_rate_index=usdclp_obs,
    fx_rate_index_fixing_lag=2,
    is_bond=es_bono
)

In [17]:
params = dict(
    rec_pay=rp,
    start_date=fecha_inicio,
    end_date=fecha_final,
    bus_adj_rule=bus_adj_rule,
    settlement_periodicity=periodicidad,
    stub_period=periodo_irregular,
    settlement_calendar=calendario,
    settlement_lag=lag_pago,
    initial_notional=nominal,
    amort_is_cashflow=amort_es_flujo,
    interest_rate=tasa_cupon,
    notional_currency=usd,
    settlement_currency=clp,
    fx_rate_index=usdclp_obs,
    fx_rate_index_fixing_lag=2,
    is_bond=es_bono
)

In [18]:
fixed_rate_mccy_leg = qcf.LegFactory.build_bullet_fixed_rate_mccy_leg(
    **params
)

In [19]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, fixed_rate_mccy_leg.size()):
    tabla.append(qcf.show(fixed_rate_mccy_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha__pago', 'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo', 'moneda_nominal',
            'valor_tasa', 'tipo_tasa', 'fecha_fijacion_fx', 'moneda_pago', 'indice_fx', 'valor_indice_fx', 'amort_moneda_pago',
           'interes_moneda_pago']
df4 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df4

,fecha_inicial,fecha__final,fecha__pago,nominal,amort,interes,amort_es_flujo,flujo,moneda_nominal,valor_tasa,tipo_tasa,fecha_fijacion_fx,moneda_pago,indice_fx,valor_indice_fx,amort_moneda_pago,interes_moneda_pago
0,1969-01-31,1969-07-31,1969-08-01,100000.0,0.0,1500.000000,False,1500.000000,USD,0.03,Lin30360,1969-07-30,CLP,USDOBS,1.0,0.0,1500.000000
1,1969-07-31,1970-01-30,1970-02-02,100000.0,0.0,1500.000000,False,1500.000000,USD,0.03,Lin30360,1970-01-29,CLP,USDOBS,1.0,0.0,1500.000000
2,1970-01-30,1970-07-31,1970-08-03,100000.0,0.0,1500.000000,False,1500.000000,USD,0.03,Lin30360,1970-07-30,CLP,USDOBS,1.0,0.0,1500.000000
3,1970-07-31,1971-01-29,1971-02-01,100000.0,0.0,1491.666667,False,1491.666667,USD,0.03,Lin30360,1971-01-28,CLP,USDOBS,1.0,0.0,1491.666667
4,1971-01-29,1971-07-30,1971-08-02,100000.0,0.0,1508.333333,False,1508.333333,USD,0.03,Lin30360,1971-07-29,CLP,USDOBS,1.0,0.0,1508.333333
5,1971-07-30,1972-01-31,1972-02-01,100000.0,0.0,1500.000000,False,1500.000000,USD,0.03,Lin30360,1972-01-28,CLP,USDOBS,1.0,0.0,1500.000000
6,1972-01-31,1972-07-31,1972-08-01,100000.0,0.0,1500.000000,False,1500.000000,USD,0.03,Lin30360,1972-07-28,CLP,USDOBS,1.0,0.0,1500.000000
7,1972-07-31,1973-01-31,1973-02-01,100000.0,0.0,1500.000000,False,1500.000000,USD,0.03,Lin30360,1973-01-30,CLP,USDOBS,1.0,0.0,1500.000000
8,1973-01-31,1973-07-31,1973-08-01,100000.0,0.0,1500.000000,False,1500.000000,USD,0.03,Lin30360,1973-07-30,CLP,USDOBS,1.0,0.0,1500.000000
9,1973-07-31,1974-01-31,1974-02-01,100000.0,100000.0,1500.000000,False,1500.000000,USD,0.03,Lin30360,1974-01-30,CLP,USDOBS,1.0,100000.0,1500.000000


### Construcción Asistida de un `CustomAmortFixedRateMultiCurrencyLeg`
Se verá como construir objetos `Leg` donde cada `Cashflow` es un objeto de tipo `FixedRateMultiCurrencyCashflow`, todos con la misma tasa fija. En el primer ejemplo se construye un `Leg` de tipo *bullet*: una única amortización igual al capital vigente de todos los `FixedRateMultiCurrencyCasflow` en el último flujo.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `CustomNotionalAndAmort`: vector de capital vigente y amortizaciones customizado
- `bool`: si es `True` significa que la amortización es un flujo de caja efectivo
- `QCInterestRate`: la tasa a aplicar en cada flujo
- `QCCurrency`: moneda del nominal
- `QCCurrency`: moneda de los flujos
- `FXRateIndex`: índice con el cual se transforma cada flujo a la moneda de pago.
- `bool`: si es `True` fuerza a que las fechas de pago coincidan con las fechas finales. Esto para lograr una valorización acorde a las convenciones de los mercados de renta fija, en caso que la `Leg` represente un bono a tasa fija.

Vamos a un ejemplo. Cambiando los parámetros siguientes se puede visualizar el efecto de ellos en la construcción.

In [20]:
# Primero se debe dar de alta un FXRateIndex
usd = qcf.QCUSD()
clp = qcf.QCCLP()
usdclp = qcf.FXRate(usd, clp)
one_d = qcf.Tenor('1D')
usdclp_obs = qcf.FXRateIndex(usdclp, 'USDOBS', one_d, one_d, calendario)

# Luego se dan de alta los otros parámetros requeridos para la construcción
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(31, 1, 1969)
fecha_final = qcf.QCDate(31, 1, 1974) 
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad = qcf.Tenor('6M')
periodo_irregular = qcf.StubPeriod.NO
lag_pago = 1
es_bono = False

# Amortizaciones
custom_notional_amort = qcf.CustomNotionalAmort()
custom_notional_amort.set_size(10)
for i in range(0, 10):
    custom_notional_amort.set_notional_amort_at(i, 1000.0 - i * 100.0, 100.0)

# Se da de alta el objeto
fixed_rate_mccy_leg = qcf.LegFactory.build_custom_amort_fixed_rate_mccy_leg(
    rec_pay=rp,
    start_date=fecha_inicio,
    end_date=fecha_final,
    bus_adj_rule=bus_adj_rule,
    settlement_periodicity=periodicidad,
    stub_period=periodo_irregular,
    settlement_calendar=calendario,
    settlement_lag=lag_pago,
    notional_and_amort=custom_notional_amort,
    amort_is_cashflow=amort_es_flujo,
    interest_rate=tasa_cupon,
    notional_currency=usd,
    settlement_currency=clp,
    fx_rate_index=usdclp_obs,
    fx_rate_index_fixing_lag=2,
    is_bond=es_bono
)

In [21]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, fixed_rate_mccy_leg.size()):
    tabla.append(qcf.show(fixed_rate_mccy_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha__pago', 'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo', 'moneda_nominal',
            'valor_tasa', 'tipo_tasa', 'fecha_fijacion_fx', 'moneda_pago', 'indice_fx', 'valor_indice_fx', 'amort_moneda_pago',
           'interes_moneda_pago']
df4 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df4

,fecha_inicial,fecha__final,fecha__pago,nominal,amort,interes,amort_es_flujo,flujo,moneda_nominal,valor_tasa,tipo_tasa,fecha_fijacion_fx,moneda_pago,indice_fx,valor_indice_fx,amort_moneda_pago,interes_moneda_pago
0,1969-01-31,1969-07-31,1969-08-01,1000.0,100.0,15.000000,False,15.000000,USD,0.03,Lin30360,1969-07-30,CLP,USDOBS,1.0,100.0,15.000000
1,1969-07-31,1970-01-30,1970-02-02,900.0,100.0,13.500000,False,13.500000,USD,0.03,Lin30360,1970-01-29,CLP,USDOBS,1.0,100.0,13.500000
2,1970-01-30,1970-07-31,1970-08-03,800.0,100.0,12.000000,False,12.000000,USD,0.03,Lin30360,1970-07-30,CLP,USDOBS,1.0,100.0,12.000000
3,1970-07-31,1971-01-29,1971-02-01,700.0,100.0,10.441667,False,10.441667,USD,0.03,Lin30360,1971-01-28,CLP,USDOBS,1.0,100.0,10.441667
4,1971-01-29,1971-07-30,1971-08-02,600.0,100.0,9.050000,False,9.050000,USD,0.03,Lin30360,1971-07-29,CLP,USDOBS,1.0,100.0,9.050000
5,1971-07-30,1972-01-31,1972-02-01,500.0,100.0,7.500000,False,7.500000,USD,0.03,Lin30360,1972-01-28,CLP,USDOBS,1.0,100.0,7.500000
6,1972-01-31,1972-07-31,1972-08-01,400.0,100.0,6.000000,False,6.000000,USD,0.03,Lin30360,1972-07-28,CLP,USDOBS,1.0,100.0,6.000000
7,1972-07-31,1973-01-31,1973-02-01,300.0,100.0,4.500000,False,4.500000,USD,0.03,Lin30360,1973-01-30,CLP,USDOBS,1.0,100.0,4.500000
8,1973-01-31,1973-07-31,1973-08-01,200.0,100.0,3.000000,False,3.000000,USD,0.03,Lin30360,1973-07-30,CLP,USDOBS,1.0,100.0,3.000000
9,1973-07-31,1974-01-31,1974-02-01,100.0,100.0,1.500000,False,1.500000,USD,0.03,Lin30360,1974-01-30,CLP,USDOBS,1.0,100.0,1.500000


### Construcción Asistida de un `BulletIborLeg`
En este ejemplo se construye un `Leg` con `IborCashflow` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `Tenor`: periodicidad de fijación
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular para el calendario de fijaciones
- `QCBusinessCalendar`: calendario que aplica para las fechas de fijación
- `unsigned int`: lag de fijación expresado en días
- `InterestRateIndex`: índice de tasa de interés utilizado en cada `IborCashflow`
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `QCCurrency`: moneda del nominal y de los flujos
- `float`: spread aditivo
- `gearing`: spread multiplicativo

Vamos a un ejemplo. Cambiando los parámetros anteriores se puede visualizar el efecto de ellos en la construcción. 

**NOTA:** para construir un `Leg` con `IborCashflow` y amortización customizada, sólo se debe cambiar el parámetro **nominal** por **CustomNotionalAndAmort** e invocar el método `qcf.LegFactory.build_custom_amort_ibor_leg(...)`.

In [22]:
# Se da de alta los parámetros requeridos
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(31, 1, 1969)
fecha_final = qcf.QCDate(31, 1, 1974) 
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = qcf.Tenor('3M')
periodo_irregular_pago = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
periodicidad_fijacion = qcf.Tenor('3M')
periodo_irregular_fijacion = qcf.StubPeriod.NO

# vamos a usar el mismo calendario para pago y fijaciones
lag_de_fijacion = 2

# Definición del índice
codigo = 'LIBORUSD3M'
lin_act360 = qcf.QCInterestRate(.0, qcf.QCAct360(), qcf.QCLinearWf())
fixing_lag = qcf.Tenor('2d')
tenor = qcf.Tenor('3m')
fixing_calendar = calendario
settlement_calendar = calendario
usd = qcf.QCUSD()
libor_usd_3m = qcf.InterestRateIndex(
    codigo,
    lin_act360,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    usd
)
# Fin índice

nominal = 1000000.0
amort_es_flujo = True 
moneda = usd
spread = .01
gearing = 1.0

ibor_leg = qcf.LegFactory.build_bullet_ibor_leg(
    rec_pay=rp, 
    start_date=fecha_inicio, 
    end_date=fecha_final, 
    bus_adj_rule=bus_adj_rule, 
    settlement_periodicity=periodicidad_pago,
    stub_period=periodo_irregular_pago, 
    settlement_calendar=calendario, 
    settlement_lag=lag_pago,
    fixing_periodicity=periodicidad_fijacion, 
    fixing_stub_period=periodo_irregular_fijacion,
    fixing_calendar=calendario, 
    fixing_lag=lag_de_fijacion, 
    interest_rate_index=libor_usd_3m,
    initial_notional=nominal, 
    amort_is_cashflow=amort_es_flujo, 
    notional_currency=moneda, 
    spread=spread, 
    gearing=gearing,
)

In [23]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, ibor_leg.size()):
    tabla.append(qcf.show(ibor_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha_fixing', 'fecha__pago', 'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo',
            'moneda', 'codigo_indice', 'valor_tasa', 'spread', 'gearing', 'tipo_tasa']
df5 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df5

,fecha_inicial,fecha__final,fecha_fixing,fecha__pago,nominal,amort,interes,amort_es_flujo,flujo,moneda,codigo_indice,valor_tasa,spread,gearing,tipo_tasa
0,1969-01-31,1969-04-30,1969-01-29,1969-04-30,1000000.0,0.0,2472.222222,True,2.472222e+03,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
1,1969-04-30,1969-07-31,1969-04-28,1969-07-31,1000000.0,0.0,2555.555556,True,2.555556e+03,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
2,1969-07-31,1969-10-31,1969-07-29,1969-10-31,1000000.0,0.0,2555.555556,True,2.555556e+03,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
3,1969-10-31,1970-01-30,1969-10-29,1970-01-30,1000000.0,0.0,2527.777778,True,2.527778e+03,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
4,1970-01-30,1970-04-30,1970-01-28,1970-04-30,1000000.0,0.0,2500.000000,True,2.500000e+03,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
5,1970-04-30,1970-07-31,1970-04-28,1970-07-31,1000000.0,0.0,2555.555556,True,2.555556e+03,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
6,1970-07-31,1970-10-30,1970-07-29,1970-10-30,1000000.0,0.0,2527.777778,True,2.527778e+03,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
7,1970-10-30,1971-01-29,1970-10-28,1971-01-29,1000000.0,0.0,2527.777778,True,2.527778e+03,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
8,1971-01-29,1971-04-30,1971-01-27,1971-04-30,1000000.0,0.0,2527.777778,True,2.527778e+03,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
9,1971-04-30,1971-07-30,1971-04-28,1971-07-30,1000000.0,0.0,2527.777778,True,2.527778e+03,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360


Ahora con amortización customizada.

In [24]:
# Amortizaciones
custom_notional_amort = qcf.CustomNotionalAmort()
custom_notional_amort.set_size(20)
for i in range(0, 20):
    custom_notional_amort.set_notional_amort_at(i, 2000.0 - i * 100.0, 100.0)

In [25]:
custom_amort_ibor_leg = qcf.LegFactory.build_custom_amort_ibor_leg(
    rec_pay=rp, 
    start_date=fecha_inicio, 
    end_date=fecha_final, 
    bus_adj_rule=bus_adj_rule, 
    settlement_periodicity=periodicidad_pago,
    stub_period=periodo_irregular_pago, 
    settlement_calendar=calendario, 
    settlement_lag=lag_pago,
    fixing_periodicity=periodicidad_fijacion, 
    fixing_stub_period=periodo_irregular_fijacion,
    fixing_calendar=calendario, 
    fixing_lag=lag_de_fijacion, 
    interest_rate_index=libor_usd_3m,
    notional_and_amort=custom_notional_amort, 
    amort_is_cashflow=amort_es_flujo, 
    notional_currency=moneda, 
    spread=spread, 
    gearing=gearing,
)

In [26]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, custom_amort_ibor_leg.size()):
    tabla.append(qcf.show(custom_amort_ibor_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha_fixing', 'fecha__pago', 'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo',
            'moneda', 'codigo_indice', 'valor_tasa', 'spread', 'gearing', 'tipo_tasa']
df5 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato 
df5

,fecha_inicial,fecha__final,fecha_fixing,fecha__pago,nominal,amort,interes,amort_es_flujo,flujo,moneda,codigo_indice,valor_tasa,spread,gearing,tipo_tasa
0,1969-01-31,1969-04-30,1969-01-29,1969-04-30,2000.0,100.0,0.002472,True,100.002472,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
1,1969-04-30,1969-07-31,1969-04-28,1969-07-31,1900.0,100.0,0.002556,True,100.002556,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
2,1969-07-31,1969-10-31,1969-07-29,1969-10-31,1800.0,100.0,0.002556,True,100.002556,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
3,1969-10-31,1970-01-30,1969-10-29,1970-01-30,1700.0,100.0,0.002528,True,100.002528,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
4,1970-01-30,1970-04-30,1970-01-28,1970-04-30,1600.0,100.0,0.002500,True,100.002500,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
5,1970-04-30,1970-07-31,1970-04-28,1970-07-31,1500.0,100.0,0.002556,True,100.002556,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
6,1970-07-31,1970-10-30,1970-07-29,1970-10-30,1400.0,100.0,0.002528,True,100.002528,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
7,1970-10-30,1971-01-29,1970-10-28,1971-01-29,1300.0,100.0,0.002528,True,100.002528,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
8,1971-01-29,1971-04-30,1971-01-27,1971-04-30,1200.0,100.0,0.002528,True,100.002528,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
9,1971-04-30,1971-07-30,1971-04-28,1971-07-30,1100.0,100.0,0.002528,True,100.002528,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360


### Construcción Asistida de un `BulletIborMultiCurrencyLeg`
En este ejemplo se construye un `Leg` con `IborMultiCurrencyCashflow` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `Tenor`: periodicidad de fijación
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular para el calendario de fijaciones
- `QCBusinessCalendar`: calendario que aplica para las fechas de fijación
- `unsigned int`: lag de fijación expresado en días
- `InterestRateIndex`: índice de tasa de interés utilizado en cada `IborMultiCurrencyCashflow`
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `QCCurrency`: moneda del nominal
- `float`: spread aditivo
- `gearing`: spread multiplicativo
- `QCCurrency`: moneda del nominal
- `QCCurrency`: moneda de pago los flujos
- `FXRateIndex`: índice con el cual se transforma cada flujo a la moneda de pago
- `int`: lag de fijación del FXRateIndex (respecto a settlement date)

Vamos a un ejemplo. Cambiando los parámetros anteriores se puede visualizar el efecto de ellos en la construcción. 

In [27]:
ibor_mccy_leg = qcf.LegFactory.build_bullet_ibor_mccy_leg(
    rec_pay=rp, 
    start_date=fecha_inicio, 
    end_date=fecha_final, 
    bus_adj_rule=bus_adj_rule, 
    settlement_periodicity=periodicidad_pago,
    stub_period=periodo_irregular_pago, 
    settlement_calendar=calendario, 
    settlement_lag=1, #lag_pago,
    fixing_periodicity=periodicidad_fijacion, 
    fixing_stub_period=periodo_irregular_fijacion,
    fixing_calendar=calendario, 
    fixing_lag=lag_de_fijacion, 
    interest_rate_index=libor_usd_3m,
    initial_notional=nominal, 
    amort_is_cashflow=amort_es_flujo, 
    notional_currency=usd, 
    spread=spread, 
    gearing=gearing,
    settlement_currency=clp, 
    fx_rate_index=usdclp_obs, 
    fx_rate_index_fixing_lag=1
)

In [28]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, ibor_mccy_leg.size()):
    ibor_mccy_leg.get_cashflow_at(i).set_fx_rate_index_value(10.0)
    tabla.append(qcf.show(ibor_mccy_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha_fixing', 'fecha__pago',
            'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo',
            'moneda_nominal', 'codigo_indice_tasa', 'spread', 'gearing', 'valor_tasa', 'tipo_tasa',
            'fecha_fijacion_fx', 'moneda_pago', 'codigo_indice_fx', 'valor_indice_fx',
            'amort_moneda_pago', 'interes_moneda_pago']
df6 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df6.style.format(format_dict)

,fecha_inicial,fecha__final,fecha_fixing,fecha__pago,nominal,amort,interes,amort_es_flujo,flujo,moneda_nominal,codigo_indice_tasa,spread,gearing,valor_tasa,tipo_tasa,fecha_fijacion_fx,moneda_pago,codigo_indice_fx,valor_indice_fx,amort_moneda_pago,interes_moneda_pago
0,1969-01-31,1969-04-30,1969-01-29,1969-05-01,"1,000,000.00",0.00,"2,472.22",True,"24,722.22",USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1969-04-30,CLP,USDOBS,10.00,0.00,"24,722.22"
1,1969-04-30,1969-07-31,1969-04-28,1969-08-01,"1,000,000.00",0.00,"2,555.56",True,"25,555.56",USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1969-07-31,CLP,USDOBS,10.00,0.00,"25,555.56"
2,1969-07-31,1969-10-31,1969-07-29,1969-11-03,"1,000,000.00",0.00,"2,555.56",True,"25,555.56",USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1969-10-31,CLP,USDOBS,10.00,0.00,"25,555.56"
3,1969-10-31,1970-01-30,1969-10-29,1970-02-02,"1,000,000.00",0.00,"2,527.78",True,"25,277.78",USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1970-01-30,CLP,USDOBS,10.00,0.00,"25,277.78"
4,1970-01-30,1970-04-30,1970-01-28,1970-05-01,"1,000,000.00",0.00,"2,500.00",True,"25,000.00",USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1970-04-30,CLP,USDOBS,10.00,0.00,"25,000.00"
5,1970-04-30,1970-07-31,1970-04-28,1970-08-03,"1,000,000.00",0.00,"2,555.56",True,"25,555.56",USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1970-07-31,CLP,USDOBS,10.00,0.00,"25,555.56"
6,1970-07-31,1970-10-30,1970-07-29,1970-11-02,"1,000,000.00",0.00,"2,527.78",True,"25,277.78",USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1970-10-30,CLP,USDOBS,10.00,0.00,"25,277.78"
7,1970-10-30,1971-01-29,1970-10-28,1971-02-01,"1,000,000.00",0.00,"2,527.78",True,"25,277.78",USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1971-01-29,CLP,USDOBS,10.00,0.00,"25,277.78"
8,1971-01-29,1971-04-30,1971-01-27,1971-05-03,"1,000,000.00",0.00,"2,527.78",True,"25,277.78",USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1971-04-30,CLP,USDOBS,10.00,0.00,"25,277.78"
9,1971-04-30,1971-07-30,1971-04-28,1971-08-02,"1,000,000.00",0.00,"2,527.78",True,"25,277.78",USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1971-07-30,CLP,USDOBS,10.00,0.00,"25,277.78"


Ahora con amortización customizada.

In [29]:
ibor_custom_amort_mccy_leg = qcf.LegFactory.build_custom_amort_ibor_mccy_leg(
    rec_pay=rp, 
    start_date=fecha_inicio, 
    end_date=fecha_final, 
    bus_adj_rule=bus_adj_rule, 
    settlement_periodicity=periodicidad_pago,
    stub_period=periodo_irregular_pago, 
    settlement_calendar=calendario, 
    settlement_lag=1, #lag_pago,
    fixing_periodicity=periodicidad_fijacion, 
    fixing_stub_period=periodo_irregular_fijacion,
    fixing_calendar=calendario, 
    fixing_lag=lag_de_fijacion, 
    interest_rate_index=libor_usd_3m,
    notional_and_amort=custom_notional_amort, 
    amort_is_cashflow=amort_es_flujo, 
    notional_currency=usd, 
    spread=spread, 
    gearing=gearing,
    settlement_currency=clp, 
    fx_rate_index=usdclp_obs, 
    fx_rate_index_fixing_lag=1
)

In [30]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, ibor_custom_amort_mccy_leg.size()):
    ibor_mccy_leg.get_cashflow_at(i).set_fx_rate_index_value(10.0)
    tabla.append(qcf.show(ibor_custom_amort_mccy_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha_fixing', 'fecha__pago',
            'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo',
            'moneda_nominal', 'codigo_indice_tasa', 'spread', 'gearing', 'valor_tasa', 'tipo_tasa',
            'fecha_fijacion_fx', 'moneda_pago', 'codigo_indice_fx', 'valor_indice_fx',
            'amort_moneda_pago', 'interes_moneda_pago']
df6 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df6.style.format(format_dict)

,fecha_inicial,fecha__final,fecha_fixing,fecha__pago,nominal,amort,interes,amort_es_flujo,flujo,moneda_nominal,codigo_indice_tasa,spread,gearing,valor_tasa,tipo_tasa,fecha_fijacion_fx,moneda_pago,codigo_indice_fx,valor_indice_fx,amort_moneda_pago,interes_moneda_pago
0,1969-01-31,1969-04-30,1969-01-29,1969-05-01,"2,000.00",100.00,0.00,True,100.00,USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1969-04-30,CLP,USDOBS,1.00,100.00,0.00
1,1969-04-30,1969-07-31,1969-04-28,1969-08-01,"1,900.00",100.00,0.00,True,100.00,USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1969-07-31,CLP,USDOBS,1.00,100.00,0.00
2,1969-07-31,1969-10-31,1969-07-29,1969-11-03,"1,800.00",100.00,0.00,True,100.00,USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1969-10-31,CLP,USDOBS,1.00,100.00,0.00
3,1969-10-31,1970-01-30,1969-10-29,1970-02-02,"1,700.00",100.00,0.00,True,100.00,USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1970-01-30,CLP,USDOBS,1.00,100.00,0.00
4,1970-01-30,1970-04-30,1970-01-28,1970-05-01,"1,600.00",100.00,0.00,True,100.00,USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1970-04-30,CLP,USDOBS,1.00,100.00,0.00
5,1970-04-30,1970-07-31,1970-04-28,1970-08-03,"1,500.00",100.00,0.00,True,100.00,USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1970-07-31,CLP,USDOBS,1.00,100.00,0.00
6,1970-07-31,1970-10-30,1970-07-29,1970-11-02,"1,400.00",100.00,0.00,True,100.00,USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1970-10-30,CLP,USDOBS,1.00,100.00,0.00
7,1970-10-30,1971-01-29,1970-10-28,1971-02-01,"1,300.00",100.00,0.00,True,100.00,USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1971-01-29,CLP,USDOBS,1.00,100.00,0.00
8,1971-01-29,1971-04-30,1971-01-27,1971-05-03,"1,200.00",100.00,0.00,True,100.00,USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1971-04-30,CLP,USDOBS,1.00,100.00,0.00
9,1971-04-30,1971-07-30,1971-04-28,1971-08-02,"1,100.00",100.00,0.00,True,100.00,USD,LIBORUSD3M,1.0000%,1.00,0.0000%,LinAct360,1971-07-30,CLP,USDOBS,1.00,100.00,0.00


### Construcción Asistida de un `OvernightIndexLeg`

En este ejemplo se construye un `Leg` con `OvernightIndexCashflow` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en las fechas iniciales y finales de fijación del índice en caso de días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `QCBusinessCalendar`: calendario que aplica para las fechas de fijación de índice
- `unsigned int`: lag de pago expresado en días
- `float`: nocional inicial
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `float`: spread aditivo
- `float`: spread multiplicativo
- `QCInterestRate`: indica el tipo de tasa equivalente
- `string`: nombre del índice overnight
- `unsigned int`: número de decimales a usar en el cálculo de la tasa equivalente
- `QCCurrency`: moneda del nocional

Vamos a un ejemplo. Cambiando los parámetros anteriores se puede visualizar el efecto de ellos en la construcción. 

In [31]:
# Se da de alta los parámetros requeridos
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(12, 1, 1968)
fecha_final = qcf.QCDate(12, 1, 1971) 
bus_adj_rule = qcf.BusyAdjRules.NO
fix_adj_rule = qcf.BusyAdjRules.PREVIOUS
periodicidad_pago = qcf.Tenor('6M')
periodo_irregular_pago = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
nominal = 1000000.0
amort_es_flujo = True 
spread = .01
gearing = 1.0
rate = qcf.QCInterestRate(0.0, qcf.QCAct360(), qcf.QCLinearWf())
indice = 'INDICE'
num_decimales = 8
moneda = qcf.QCUSD()

In [32]:
overnight_index_leg = qcf.LegFactory.build_bullet_overnight_index_leg(
    rec_pay=rp,
    start_date=fecha_inicio,
    end_date=fecha_final,
    bus_adj_rule=bus_adj_rule,
    fix_adj_rule=fix_adj_rule,
    settlement_periodicity=periodicidad_pago,
    stub_period=periodo_irregular_pago,
    settlement_calendar=calendario,
    fixing_calendar=calendario,
    settlement_lag=lag_pago,
    initial_notional=nominal,
    amort_is_cashflow=amort_es_flujo,
    spread=spread,
    gearing=gearing,
    interest_rate=rate,
    index_name=indice,
    eq_rate_decimal_places=num_decimales,
    notional_currency=moneda 
)

In [33]:
c = overnight_index_leg.get_cashflow_at(0)
c.set_end_date_index(1.1)

In [34]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, overnight_index_leg.size()):
    tabla.append(qcf.show(overnight_index_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = qcf.get_column_names("OvernightIndexCashflow", "")
df7 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df7.style.format(format_dict)

,fecha_inicial_devengo,fecha_final_devengo,fecha_inicial_indice,fecha_final_indice,fecha_pago,nocional,amortizacion,amort_es_flujo,moneda_nocional,nombre_indice,valor_indice_inicial,valor_indice_final,valor_tasa_equivalente,tipo_tasa,interes,flujo,spread,gearing
0,1968-01-12,1968-07-12,1968-01-12,1968-07-12,1968-07-12,"1,000,000.00",0.00,True,USD,INDICE,1.000000,1.100000,19.780220%,LinAct360,"105,055.56","105,055.56",1.0000%,1.00
1,1968-07-12,1969-01-12,1968-07-12,1969-01-10,1969-01-13,"1,000,000.00",0.00,True,USD,INDICE,1.000000,1.000000,0.000000%,LinAct360,"5,111.11","5,111.11",1.0000%,1.00
2,1969-01-12,1969-07-12,1969-01-10,1969-07-11,1969-07-14,"1,000,000.00",0.00,True,USD,INDICE,1.000000,1.000000,0.000000%,LinAct360,"5,027.78","5,027.78",1.0000%,1.00
3,1969-07-12,1970-01-12,1969-07-11,1970-01-12,1970-01-12,"1,000,000.00",0.00,True,USD,INDICE,1.000000,1.000000,0.000000%,LinAct360,"5,111.11","5,111.11",1.0000%,1.00
4,1970-01-12,1970-07-12,1970-01-12,1970-07-10,1970-07-13,"1,000,000.00",0.00,True,USD,INDICE,1.000000,1.000000,0.000000%,LinAct360,"5,027.78","5,027.78",1.0000%,1.00
5,1970-07-12,1971-01-12,1970-07-10,1971-01-12,1971-01-12,"1,000,000.00","1,000,000.00",True,USD,INDICE,1.000000,1.000000,0.000000%,LinAct360,"5,111.11","1,005,111.11",1.0000%,1.00


Para construir una pata con amortizaciones hacemos lo siguiente:

In [35]:
custom_amort = qcf.CustomNotionalAmort(6)
for i in range(6):
    custom_amort.set_notional_amort_at(i, 600 - i * 100, 100)

In [36]:
custom_amort_overnight_index_leg = qcf.LegFactory.build_custom_amort_overnight_index_leg(
    rec_pay=rp,
    start_date=fecha_inicio,
    end_date=fecha_final,
    bus_adj_rule=bus_adj_rule,
    fix_adj_rule=fix_adj_rule,
    settlement_periodicity=periodicidad_pago,
    stub_period=periodo_irregular_pago,
    settlement_calendar=calendario,
    fixing_calendar=calendario,
    settlement_lag=lag_pago,
    notional_and_amort=custom_amort,
    amort_is_cashflow=amort_es_flujo,
    spread=spread,
    gearing=gearing,
    interest_rate=rate,
    index_name=indice,
    eq_rate_decimal_places=num_decimales,
    notional_currency=moneda 
)

In [37]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, custom_amort_overnight_index_leg.size()):
    tabla.append(qcf.show(custom_amort_overnight_index_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = qcf.get_column_names("OvernightIndexCashflow", "")
df7 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df7.style.format(format_dict)

,fecha_inicial_devengo,fecha_final_devengo,fecha_inicial_indice,fecha_final_indice,fecha_pago,nocional,amortizacion,amort_es_flujo,moneda_nocional,nombre_indice,valor_indice_inicial,valor_indice_final,valor_tasa_equivalente,tipo_tasa,interes,flujo,spread,gearing
0,1968-01-12,1968-07-12,1968-01-12,1968-07-12,1968-07-12,600.00,100.00,True,USD,INDICE,1.000000,1.000000,0.000000%,LinAct360,3.03,103.03,1.0000%,1.00
1,1968-07-12,1969-01-12,1968-07-12,1969-01-10,1969-01-13,500.00,100.00,True,USD,INDICE,1.000000,1.000000,0.000000%,LinAct360,2.56,102.56,1.0000%,1.00
2,1969-01-12,1969-07-12,1969-01-10,1969-07-11,1969-07-14,400.00,100.00,True,USD,INDICE,1.000000,1.000000,0.000000%,LinAct360,2.01,102.01,1.0000%,1.00
3,1969-07-12,1970-01-12,1969-07-11,1970-01-12,1970-01-12,300.00,100.00,True,USD,INDICE,1.000000,1.000000,0.000000%,LinAct360,1.53,101.53,1.0000%,1.00
4,1970-01-12,1970-07-12,1970-01-12,1970-07-10,1970-07-13,200.00,100.00,True,USD,INDICE,1.000000,1.000000,0.000000%,LinAct360,1.01,101.01,1.0000%,1.00
5,1970-07-12,1971-01-12,1970-07-10,1971-01-12,1971-01-12,100.00,100.00,True,USD,INDICE,1.000000,1.000000,0.000000%,LinAct360,0.51,100.51,1.0000%,1.00


### Construcción Asistida de un `OvernightIndexMultiCurrencyLeg`

En este ejemplo se construye un `Leg` con `OvernightIndexMultiCurrencyCashflow` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en las fechas iniciales y finales de fijación del índice en caso de días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `QCBusinessCalendar`: calendario que aplica para las fechas de fijación de índice
- `unsigned int`: lag de pago expresado en días
- `float`: nocional inicial
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `float`: spread aditivo
- `float`: spread multiplicativo
- `QCInterestRate`: indica el tipo de tasa equivalente
- `string`: nombre del índice overnight
- `unsigned int`: número de decimales a usar en el cálculo de la tasa equivalente
- `QCCurrency`: moneda del nocional
- `QCCurrency`: moneda de pago
- `FXRateIndex`: índice de tipo de cambio
- `unsigned int`: fx rate index fixing lag

Vamos a un ejemplo. Cambiando los parámetros siguientes se puede visualizar el efecto de ellos en la construcción. 

In [38]:
params = dict(
    rec_pay=qcf.RecPay.RECEIVE,
    start_date=qcf.QCDate(12, 1, 1968),
    end_date=qcf.QCDate(12, 1, 1971),
    bus_adj_rule=qcf.BusyAdjRules.NO,
    fix_adj_rule=qcf.BusyAdjRules.PREVIOUS,
    settlement_periodicity=qcf.Tenor('6M'),
    stub_period=qcf.StubPeriod.NO,
    settlement_calendar=qcf.BusinessCalendar(fecha_inicio, 20),
    fixing_calendar=qcf.BusinessCalendar(fecha_inicio, 20),
    settlement_lag=0,
    initial_notional=1000000.0,
    amort_is_cashflow=True,
    spread=.01,
    gearing=1.0,
    interest_rate=qcf.QCInterestRate(0.0, qcf.QCAct360(), qcf.QCLinearWf()),
    index_name='INDICE',
    eq_rate_decimal_places=8,
    notional_currency=qcf.QCUSD(),
    settlement_currency=qcf.QCCLP(),
    fx_rate_index=usdclp_obs, # definido más arriba
    fx_rate_index_fixing_lag=1,
)

In [39]:
bullet_overnight_index_mccy_leg = qcf.LegFactory.build_bullet_overnight_index_multi_currency_leg(
    **params
)

In [40]:
tabla = []
for i in range(bullet_overnight_index_mccy_leg.size()):
    tabla.append(qcf.show(bullet_overnight_index_mccy_leg.get_cashflow_at(i)))
df_1 = pd.DataFrame(tabla, columns=qcf.get_column_names("OvernightIndexMultiCurrencyCashflow", ""))
df_1

,fecha_inicial_devengo,fecha_final_devengo,fecha_inicial_indice,fecha_final_indice,fecha_pago,nocional,amortizacion,amort_es_flujo,moneda_nocional,nombre_indice,...,flujo,spread,gearing,moneda_pago,indice_fx,fecha_fijacion_indice_fx,valor_indice_fx,interes_moneda_pago,amortizacion_moneda_pago,flujo_moneda_pago
0,1968-01-12,1968-07-12,1968-01-12,1968-07-12,1968-07-12,1000000.0,0.0,True,USD,INDICE,...,5.055556e+03,0.01,1.0,CLP,USDOBS,1968-07-11,1.0,5055.555556,0.0,5.055556e+03
1,1968-07-12,1969-01-12,1968-07-12,1969-01-10,1969-01-13,1000000.0,0.0,True,USD,INDICE,...,5.111111e+03,0.01,1.0,CLP,USDOBS,1969-01-10,1.0,5111.111111,0.0,5.111111e+03
2,1969-01-12,1969-07-12,1969-01-10,1969-07-11,1969-07-14,1000000.0,0.0,True,USD,INDICE,...,5.027778e+03,0.01,1.0,CLP,USDOBS,1969-07-11,1.0,5027.777778,0.0,5.027778e+03
3,1969-07-12,1970-01-12,1969-07-11,1970-01-12,1970-01-12,1000000.0,0.0,True,USD,INDICE,...,5.111111e+03,0.01,1.0,CLP,USDOBS,1970-01-09,1.0,5111.111111,0.0,5.111111e+03
4,1970-01-12,1970-07-12,1970-01-12,1970-07-10,1970-07-13,1000000.0,0.0,True,USD,INDICE,...,5.027778e+03,0.01,1.0,CLP,USDOBS,1970-07-10,1.0,5027.777778,0.0,5.027778e+03
5,1970-07-12,1971-01-12,1970-07-10,1971-01-12,1971-01-12,1000000.0,1000000.0,True,USD,INDICE,...,1.005111e+06,0.01,1.0,CLP,USDOBS,1971-01-11,1.0,5111.111111,1000000.0,1.005111e+06


Veamos ahora el caso Multi Currency.

In [53]:
custom_amort = qcf.CustomNotionalAmort(6)
for i in range(6):
    custom_amort.set_notional_amort_at(i, 600 - i * 100, 100)

In [54]:
params = dict(
    rec_pay=qcf.RecPay.RECEIVE,
    start_date=qcf.QCDate(12, 1, 1968),
    end_date=qcf.QCDate(12, 1, 1971),
    bus_adj_rule=qcf.BusyAdjRules.NO,
    fix_adj_rule=qcf.BusyAdjRules.PREVIOUS,
    settlement_periodicity=qcf.Tenor('6M'),
    stub_period=qcf.StubPeriod.NO,
    settlement_calendar=qcf.BusinessCalendar(fecha_inicio, 20),
    fixing_calendar=qcf.BusinessCalendar(fecha_inicio, 20),
    settlement_lag=0,
    notional_and_amort=custom_amort,
    amort_is_cashflow=True,
    spread=.01,
    gearing=1.0,
    interest_rate=qcf.QCInterestRate(0.0, qcf.QCAct360(), qcf.QCLinearWf()),
    index_name='INDICE',
    eq_rate_decimal_places=8,
    notional_currency=qcf.QCUSD(),
    settlement_currency=qcf.QCCLP(),
    fx_rate_index=usdclp_obs, # definido más arriba
    fx_rate_index_fixing_lag=1,
)

In [56]:
custom_overnight_index_mccy_leg = qcf.LegFactory.build_custom_amort_overnight_index_multi_currency_leg(
    **params
)

In [57]:
tabla = []
for i in range(custom_overnight_index_mccy_leg.size()):
    tabla.append(qcf.show(custom_overnight_index_mccy_leg.get_cashflow_at(i)))
df_1 = pd.DataFrame(tabla, columns=qcf.get_column_names("OvernightIndexMultiCurrencyCashflow", ""))
df_1

,fecha_inicial_devengo,fecha_final_devengo,fecha_inicial_indice,fecha_final_indice,fecha_pago,nocional,amortizacion,amort_es_flujo,moneda_nocional,nombre_indice,...,flujo,spread,gearing,moneda_pago,indice_fx,fecha_fijacion_indice_fx,valor_indice_fx,interes_moneda_pago,amortizacion_moneda_pago,flujo_moneda_pago
0,1968-01-12,1968-07-12,1968-01-12,1968-07-12,1968-07-12,600.0,100.0,True,USD,INDICE,...,103.033333,0.01,1.0,CLP,USDOBS,1968-07-11,1.0,3.033333,100.0,103.033333
1,1968-07-12,1969-01-12,1968-07-12,1969-01-10,1969-01-13,500.0,100.0,True,USD,INDICE,...,102.555556,0.01,1.0,CLP,USDOBS,1969-01-10,1.0,2.555556,100.0,102.555556
2,1969-01-12,1969-07-12,1969-01-10,1969-07-11,1969-07-14,400.0,100.0,True,USD,INDICE,...,102.011111,0.01,1.0,CLP,USDOBS,1969-07-11,1.0,2.011111,100.0,102.011111
3,1969-07-12,1970-01-12,1969-07-11,1970-01-12,1970-01-12,300.0,100.0,True,USD,INDICE,...,101.533333,0.01,1.0,CLP,USDOBS,1970-01-09,1.0,1.533333,100.0,101.533333
4,1970-01-12,1970-07-12,1970-01-12,1970-07-10,1970-07-13,200.0,100.0,True,USD,INDICE,...,101.005556,0.01,1.0,CLP,USDOBS,1970-07-10,1.0,1.005556,100.0,101.005556
5,1970-07-12,1971-01-12,1970-07-10,1971-01-12,1971-01-12,100.0,100.0,True,USD,INDICE,...,100.511111,0.01,1.0,CLP,USDOBS,1971-01-11,1.0,0.511111,100.0,100.511111


### Construcción Asistida de un `IcpClpLeg`
En este ejemplo se construye un `Leg` con `IcpClpCashflow` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `float`: spread aditivo
- `gearing`: spread multiplicativo

Vamos a un ejemplo. Cambiando los parámetros anteriores se puede visualizar el efecto de ellos en la construcción. 

**NOTA:** para construir un `Leg` con `IcpClpCashflow` y amortización customizada, sólo se debe cambiar el parámetro **nominal** por **CustomNotionalAndAmort** e invocar el método `qcf.LegFactory.build_custom_amort_icp_clp_leg(...)`.

In [41]:
# Se da de alta los parámetros requeridos
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(31, 1, 1969)
fecha_final = qcf.QCDate(31, 1, 1974) 
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = qcf.Tenor('3M')
periodo_irregular_pago = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
nominal = 1000000.0
amort_es_flujo = True 
spread = .01
gearing = 1.0

icp_clp_leg = qcf.LegFactory.build_bullet_icp_clp_leg(
    rp, 
    fecha_inicio, 
    fecha_final, 
    bus_adj_rule, 
    periodicidad_pago,
    periodo_irregular_pago, 
    calendario, 
    lag_pago,
    nominal, 
    amort_es_flujo, 
    spread, 
    gearing
)

In [42]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, icp_clp_leg.size()):
    tabla.append(qcf.show(icp_clp_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha__pago', 'nominal', 'amort', 'amort_es_flujo', 'flujo',
            'moneda', 'icp_inicial', 'icp_final', 'valor_tasa', 'interes', 'spread', 'gearing', 'tipo_tasa']
df7 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df7.style.format(format_dict)

,fecha_inicial,fecha__final,fecha__pago,nominal,amort,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,1969-01-31,1969-04-30,1969-04-30,"1,000,000.00",0.00,True,"2,472.22",CLP,"10,000.00","10,000.00",0.0000%,"2,472.22",1.0000%,1.00,LinAct360
1,1969-04-30,1969-07-31,1969-07-31,"1,000,000.00",0.00,True,"2,555.56",CLP,"10,000.00","10,000.00",0.0000%,"2,555.56",1.0000%,1.00,LinAct360
2,1969-07-31,1969-10-31,1969-10-31,"1,000,000.00",0.00,True,"2,555.56",CLP,"10,000.00","10,000.00",0.0000%,"2,555.56",1.0000%,1.00,LinAct360
3,1969-10-31,1970-01-30,1970-01-30,"1,000,000.00",0.00,True,"2,527.78",CLP,"10,000.00","10,000.00",0.0000%,"2,527.78",1.0000%,1.00,LinAct360
4,1970-01-30,1970-04-30,1970-04-30,"1,000,000.00",0.00,True,"2,500.00",CLP,"10,000.00","10,000.00",0.0000%,"2,500.00",1.0000%,1.00,LinAct360
5,1970-04-30,1970-07-31,1970-07-31,"1,000,000.00",0.00,True,"2,555.56",CLP,"10,000.00","10,000.00",0.0000%,"2,555.56",1.0000%,1.00,LinAct360
6,1970-07-31,1970-10-30,1970-10-30,"1,000,000.00",0.00,True,"2,527.78",CLP,"10,000.00","10,000.00",0.0000%,"2,527.78",1.0000%,1.00,LinAct360
7,1970-10-30,1971-01-29,1971-01-29,"1,000,000.00",0.00,True,"2,527.78",CLP,"10,000.00","10,000.00",0.0000%,"2,527.78",1.0000%,1.00,LinAct360
8,1971-01-29,1971-04-30,1971-04-30,"1,000,000.00",0.00,True,"2,527.78",CLP,"10,000.00","10,000.00",0.0000%,"2,527.78",1.0000%,1.00,LinAct360
9,1971-04-30,1971-07-30,1971-07-30,"1,000,000.00",0.00,True,"2,527.78",CLP,"10,000.00","10,000.00",0.0000%,"2,527.78",1.0000%,1.00,LinAct360


### Construcción Asistida de un `IcpClp2Leg`


En este ejemplo se construye un `Leg` con `IcpClpCashflow2` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `float`: spread aditivo
- `gearing`: spread multiplicativo

Vamos a un ejemplo. Cambiando los parámetros anteriores se puede visualizar el efecto de ellos en la construcción. 

**NOTA:** para construir un `Leg` con `IcpClpCashflow2` y amortización customizada, sólo se debe cambiar el parámetro **nominal** por **CustomNotionalAndAmort** e invocar el método `qcf.LegFactory.build_custom_amort_icp_clp2_leg(...)`.

In [43]:
# Se da de alta los parámetros requeridos
icp_clp2_leg = qcf.LegFactory.build_bullet_icp_clp2_leg(
    rp, 
    fecha_inicio:=qcf.QCDate(26, 3, 2019), 
    fecha_final:=qcf.QCDate(26, 3, 2025), 
    bus_adj_rule, 
    periodicidad_pago:=qcf.Tenor("6M"),
    periodo_irregular_pago, 
    calendario, 
    lag_pago,
    nominal, 
    amort_es_flujo, 
    spread:=0, 
    gearing, 
    True
)

In [44]:
bus_adj_rule

<BusyAdjRules.MODFOLLOW: 2>

In [45]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, icp_clp2_leg.size()):
    tabla.append(qcf.show(icp_clp2_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha__pago', 'nominal', 'amort', 'amort_es_flujo', 'flujo',
            'moneda', 'icp_inicial', 'icp_final', 'valor_tasa', 'interes', 'spread', 'gearing', 'tipo_tasa']
df9 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df9.style.format(format_dict)

,fecha_inicial,fecha__final,fecha__pago,nominal,amort,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2019-03-26,2019-09-26,2019-09-26,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
1,2019-09-26,2020-03-26,2020-03-26,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
2,2020-03-26,2020-09-28,2020-09-28,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
3,2020-09-28,2021-03-26,2021-03-26,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
4,2021-03-26,2021-09-27,2021-09-27,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
5,2021-09-27,2022-03-28,2022-03-28,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
6,2022-03-28,2022-09-26,2022-09-26,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
7,2022-09-26,2023-03-27,2023-03-27,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
8,2023-03-27,2023-09-26,2023-09-26,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
9,2023-09-26,2024-03-26,2024-03-26,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360


Se puede customizar la amortización de la siguiente forma:

In [46]:
cna = qcf.CustomNotionalAmort()
cna.set_size(19)

In [47]:
cna.set_notional_amort_at(0, 1000000, 0)
for i in range(1, icp_clp2_leg.size()):
    prev_amort = cna.get_amort_at(i - 1)
    prev_notional = cna.get_notional_at(i - 1)
    if i == 10 or i == icp_clp2_leg.size() - 1:
        cna.set_notional_amort_at(i, prev_notional - prev_amort, 500000)
    else:
        cna.set_notional_amort_at(i, prev_notional - prev_amort, 0)

for i in range(icp_clp2_leg.size()):
    cshflw = icp_clp2_leg.get_cashflow_at(i)
    cshflw.set_nominal(cna.get_notional_at(i))
    cshflw.set_amortization(cna.get_amort_at(i))

In [48]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, icp_clp2_leg.size()):
    tabla.append(qcf.show(icp_clp2_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha__pago', 'nominal', 'amort', 'amort_es_flujo', 'flujo',
            'moneda', 'icp_inicial', 'icp_final', 'valor_tasa', 'interes', 'spread', 'gearing', 'tipo_tasa']
df9 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df9.style.format(format_dict)

,fecha_inicial,fecha__final,fecha__pago,nominal,amort,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2019-03-26,2019-09-26,2019-09-26,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
1,2019-09-26,2020-03-26,2020-03-26,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
2,2020-03-26,2020-09-28,2020-09-28,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
3,2020-09-28,2021-03-26,2021-03-26,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
4,2021-03-26,2021-09-27,2021-09-27,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
5,2021-09-27,2022-03-28,2022-03-28,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
6,2022-03-28,2022-09-26,2022-09-26,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
7,2022-09-26,2023-03-27,2023-03-27,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
8,2023-03-27,2023-09-26,2023-09-26,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
9,2023-09-26,2024-03-26,2024-03-26,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360


### Construcción Asistida de un `IcpClfLeg`
En este ejemplo se construye un `Leg` con `IcpClfCashflow` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `float`: spread aditivo
- `gearing`: spread multiplicativo

Vamos a un ejemplo. Cambiando los parámetros anteriores se puede visualizar el efecto de ellos en la construcción. 

**NOTA:** para construir un `Leg` con `IcpClfCashflow` y amortización customizada, sólo se debe cambiar el parámetro **nominal** por **CustomNotionalAndAmort** e invocar el método `qcf.LegFactory.build_custom_amort_icp_clf_leg(...)`.

In [49]:
# Se da de alta los parámetros requeridos
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(31, 1, 1969)
fecha_final = qcf.QCDate(31, 1, 1974)
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = qcf.Tenor('3M')
periodo_irregular_pago = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
nominal = 1000000.0
amort_es_flujo = True
spread = .01
gearing = 1.0

icp_clf_leg = qcf.LegFactory.build_bullet_icp_clf_leg(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad_pago,
    periodo_irregular_pago,
    calendario,
    lag_pago,
    nominal,
    amort_es_flujo,
    spread,
    gearing
)

In [50]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, icp_clf_leg.size()):
    tabla.append(qcf.show(icp_clf_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha__pago', 'nominal', 'amort', 'amort_es_flujo', 'flujo',
            'moneda', 'icp_inicial', 'icp_final', 'uf_inicial', 'uf_final',
            'valor_tasa', 'interes', 'spread', 'gearing', 'tipo_tasa']
df8 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df8.style.format(format_dict)

,fecha_inicial,fecha__final,fecha__pago,nominal,amort,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,uf_inicial,uf_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,1969-01-31,1969-04-30,1969-04-30,"1,000,000.00",0.00,True,"2,472.22",CLF,"10,000.00","10,000.00",35000.000000,35000.000000,1.0000%,"2,472.22",1.0000%,1.00,LinAct360
1,1969-04-30,1969-07-31,1969-07-31,"1,000,000.00",0.00,True,"2,555.56",CLF,"10,000.00","10,000.00",35000.000000,35000.000000,1.0000%,"2,555.56",1.0000%,1.00,LinAct360
2,1969-07-31,1969-10-31,1969-10-31,"1,000,000.00",0.00,True,"2,555.56",CLF,"10,000.00","10,000.00",35000.000000,35000.000000,1.0000%,"2,555.56",1.0000%,1.00,LinAct360
3,1969-10-31,1970-01-30,1970-01-30,"1,000,000.00",0.00,True,"2,527.78",CLF,"10,000.00","10,000.00",35000.000000,35000.000000,1.0000%,"2,527.78",1.0000%,1.00,LinAct360
4,1970-01-30,1970-04-30,1970-04-30,"1,000,000.00",0.00,True,"2,500.00",CLF,"10,000.00","10,000.00",35000.000000,35000.000000,1.0000%,"2,500.00",1.0000%,1.00,LinAct360
5,1970-04-30,1970-07-31,1970-07-31,"1,000,000.00",0.00,True,"2,555.56",CLF,"10,000.00","10,000.00",35000.000000,35000.000000,1.0000%,"2,555.56",1.0000%,1.00,LinAct360
6,1970-07-31,1970-10-30,1970-10-30,"1,000,000.00",0.00,True,"2,527.78",CLF,"10,000.00","10,000.00",35000.000000,35000.000000,1.0000%,"2,527.78",1.0000%,1.00,LinAct360
7,1970-10-30,1971-01-29,1971-01-29,"1,000,000.00",0.00,True,"2,527.78",CLF,"10,000.00","10,000.00",35000.000000,35000.000000,1.0000%,"2,527.78",1.0000%,1.00,LinAct360
8,1971-01-29,1971-04-30,1971-04-30,"1,000,000.00",0.00,True,"2,527.78",CLF,"10,000.00","10,000.00",35000.000000,35000.000000,1.0000%,"2,527.78",1.0000%,1.00,LinAct360
9,1971-04-30,1971-07-30,1971-07-30,"1,000,000.00",0.00,True,"2,527.78",CLF,"10,000.00","10,000.00",35000.000000,35000.000000,1.0000%,"2,527.78",1.0000%,1.00,LinAct360


### Construcción Asistida de un `CompoundedOvernightRateLeg`

In [51]:
# Se da de alta los parámetros requeridos
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(3, 1, 2022)
fecha_final = qcf.QCDate(3, 1, 2023)
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = qcf.Tenor('3M')
periodo_irregular_pago = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0

######################################################################
# Definición del índice

codigo = 'OISTEST'
lin_act360 = qcf.QCInterestRate(.0, qcf.QCAct360(), qcf.QCLinearWf())
fixing_lag = qcf.Tenor('0d')
tenor = qcf.Tenor('1d')
fixing_calendar = calendario
settlement_calendar = calendario
usd = qcf.QCUSD()
oistest = qcf.InterestRateIndex(
    codigo,
    lin_act360,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    usd)

# Fin índice
######################################################################

nominal = 1000000.0
amort_es_flujo = True
moneda = usd
spread = .01
gearing = 1.0

cor_leg = qcf.LegFactory.build_bullet_compounded_overnight_rate_leg(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad_pago,
    periodo_irregular_pago,
    calendario,
    lag_pago,
    calendario,
    oistest,
    nominal,
    amort_es_flujo,
    usd,
    spread,
    gearing,
    True,
    8,
    0,
    0
)

In [52]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, cor_leg.size()):
    tabla.append(qcf.show(cor_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha__pago', 'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo',
            'moneda', 'indice', 'valor_tasa', 'spread', 'gearing', 'tipo_tasa']
df9 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df9.style.format(format_dict)

,fecha_inicial,fecha__final,fecha__pago,nominal,amort,interes,amort_es_flujo,flujo,moneda,indice,valor_tasa,spread,gearing,tipo_tasa
0,2022-01-03,2022-04-04,2022-04-04,"1,000,000.00",0.00,"2,527.78",True,"2,527.78",USD,OISTEST,0.0000%,1.0000%,1.00,LinAct360
1,2022-04-04,2022-07-04,2022-07-04,"1,000,000.00",0.00,"2,527.78",True,"2,527.78",USD,OISTEST,0.0000%,1.0000%,1.00,LinAct360
2,2022-07-04,2022-10-03,2022-10-03,"1,000,000.00",0.00,"2,527.78",True,"2,527.78",USD,OISTEST,0.0000%,1.0000%,1.00,LinAct360
3,2022-10-03,2023-01-03,2023-01-03,"1,000,000.00","1,000,000.00","2,555.56",True,"1,002,555.56",USD,OISTEST,0.0000%,1.0000%,1.00,LinAct360
